In [ ]:
import random
import numpy as np
from collections import deque

In [ ]:
def getlambda0(t):
    # t está en minutos, convertir a horas:
    t = t/60
    t = t%24 ##########ojo esto##########
    # Intervalo de 0 a 5 horas, lineal de 5 a 7
    if 0 <= t < 5:
        return 5 + (7 - 5) * (t - 0) / (5 - 0)
    # Intervalo de 5 a 8 horas, lineal de 7 a 6
    elif 5 <= t < 8:
        return 7 + (6 - 7) * (t - 5) / (8 - 5)
    # Intervalo de 8 a 15 horas, lineal de 6 a 9
    elif 8 <= t < 15:
        return 6 + (9 - 6) * (t - 8) / (15 - 8)
    # Intervalo de 15 a 17 horas, lineal de 9 a 6
    elif 15 <= t < 17:
        return 9 + (6 - 9) * (t - 15) / (17 - 15)
    # Intervalo de 17 a 24 horas, lineal de 6 a 5
    elif 14 <= t <= 24:
        return 6 + (5 - 6) * (t - 17) / (24 - 17)
    # Si el tiempo está fuera del rango de 0 a 24 horas
    else:
        return 10E9  

In [ ]:
# T = 24*60 # 24h en minutos
T = 30*24*60 #un mes en minutos #pongo 1 dia

nPistas = 3
nCoches = 20
nPuertas = 50

# lambda0 = getlambda0(t)
mu11 = 10.007615178514001
sigma11 = 3.0377002745229005
a12 = 10
b12 = 15
lambda2 = 1 / 2
lambda3 = 0.1

Comprobamos que funcionen las distribuciones

In [ ]:
# Funciones auxiliares generacion de tiempos
def t_llegada(t):
    return random.expovariate(getlambda0(t))*60 #nº muy pequeños, tiene sentido?

def t_aterrizaje():
    # return np.random.normal(mu11, sigma11)
    # return random.normalvariate(mu11, sigma11) 
    while True:  # Bucle infinito hasta obtener un número válido
        resultado = np.random.normal(mu11, sigma11)
        if resultado > 0:  # Comprobar si el resultado es mayor que cero
            return resultado
        
def t_traslado(): #N1- N2
    return np.random.exponential(1/lambda2)

def t_desembarques():
    return np.random.exponential(1/lambda3)

def t_salidas():
    return np.random.uniform(a12, b12)

In [ ]:
print("Lambda al inicio de la simulación es " + str(getlambda0(0)) + 
      ", al final es " + str(getlambda0(T/2)) + ' y a la mitad es ' + str(getlambda0(T/2)))

print('Tiempo de llegada: ' + str(t_llegada(T)))
print('Tiempo de llegada: ' +   str(t_llegada(T/2)))
print('Tiempo de llegada: ' +     str(t_llegada(T)))

Variables

In [ ]:
t = 0                   # Variable reloj. tiempo asíncrono.
n1 = n2 = n3 = 0        # Variables de estado nº clientes
NLL1 = NLL2 = NLL3 = 0  # Variables contador nº llegadas
NS1 = NS2 = NS3 = 0     # Variables contador nº salidas

# Lista de tiempos de eventos
LISTA = {}
LISTA['tLL1'] = LISTA['tS1'] = LISTA['tS2'] = LISTA['tS3'] = LISTA['tSS'] = -1

# Arrays for arrival and departure times
LL1 = []
LL2 = []
LL3 = []
S1 = []
S2 = []
S3 = []

n_med_n1 = n_med_n2 = n_med_n3 = 0 #nº medio de pistas ocupadas
nmed_pistasOcupadas = 0

n_cola_aterrizaje = 0 # Contador de aviones esperando para aterrizar
pistasOcupadas = 0

colaN2 = 0

#variables de estado
todas_las_pistas_ocupadas = True
coches_disponibles = True
puertas_embarque_disponibles = True

llega un avión

In [ ]:

def rutina_aterrizaje(t_suc):
    
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje
    global esperando_a_aterrizar, colaN3, colaN4
    
    print(f'----Llega un vuelo en el minuto {t_suc}----')
    # para luego calcular numero medio de clientes
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    # Actualizo variables contadores
    n1 += 1                     # aviones en n1
    NLL1 += 1                   # numero de llegadas al n1
    LL1.append(t_suc)
    
    t = t_suc  
    # Si tengo pista libre, aterrizo
    if (pistasOcupadas < nPistas):
        Z = t_aterrizaje() #random.normalvariate(mu11, sigma11) #genero tiempo aterrizaje
        #LISTA['tS1'] = t + Z #tiempo de salida del N1, que se corresponde con el final de aterrizaje
        
        LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + Z
        pistasOcupadas+=1
        print(f'Avion finalizará aterrizaje en el instante {t+Z}')
        print(f'Hay  {pistasOcupadas} pistas ocupadas')

        #entro en cola para el traslado
        colaN2+=1
    
    else:
        #si no tengo pista, no aterrizo, me pongo en cola aterrizaje
        esperando_a_aterrizar+=1
        print(f'Todas la pistas ocupadas. {esperando_a_aterrizar} aviones esperndo aterrizar')

    #=========================================================================#

    # Genero tiempo siguente llegada -> esto tiene sentido hacerlo aqui?
    Y = t_llegada(t)
    print(f'ESTAMOS EN EL ISNTANTE {t}, queda {Y} para otro vuelo')
    # Compruebo si el siguiente avión llega antes de T
    if (t + Y < T):
        LISTA['tLL1'] = t + Y #añado otro tiempo de llegada
        print(f'El siguiente avion llegará en el instante {t+Y}')
    
    else:
        print(f'Horizonte alcanzado {t + Y} , {T}, no llegan más aviones')

finaliza el aterrizaje y espera a ser trasladado

In [ ]:
def rutina_traslado(t_suc): 
    # un coche recoge al avión y lo desaloja de la pista (salida de N1)
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_aterrizar, esperando_a_despegar
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje, coches_disponibles, nPista
    global colaN3, colaN4
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    t=t_suc #Actualizamos tº de simulación
    print(f'ESTAMOS EN EL INSTANTE {t}')
    
    pistasOcupadas-=1 #DESPEJO LA PISTA EN LA QUE ESTABA EL AVION
    
    print(f'---Ahora hay {pistasOcupadas} pistas ocupadas')
    n1 -=1 #Decrementamos el nº de aviones del nodo 1 porque el avión sale del nodo 1.
    n2 +=1 #Incrementamos el nº de aviones en nodo 2
    # LISTA['tS1'][LISTA['tS1'].index(10E9)] = t_suc
    # LISTA['tS1'] = t_suc #Registramos el instante en el que sale del nodo 1.
      
    # si hay coches disponibles -> traslado
    if coches_ocupados < nCoches:
        coches_ocupados +=1 #tambien lo puedo hacer con num vehiculos
        colaN2-=1 #quito un avion de la cola de traslado
        Z = t_traslado()
        print(f'----Comienza el traslado que durará {Z}----')
        print(f'----El avión llegará a la zona de desembarque en el instante {t + Z}----')
        print(f'----Ahora hay {nCoches - coches_ocupados} coches disponibles')
        LISTA['tS2'][LISTA['tS2'].index(10E9)] = t + Z # tiempo de salida del N2, que se corresponde con el final del traslado
        
    else: #si no hay coche disponibles 
        print('----no hay vehículos disponibles---')
        #colaN2+=1 #lo pongo a la cola de ser trasladado
    
    if colaN2 > 0: #quedan aviones en la cola, genero un nuevo tiempo para el traslado del avión en cola
        print(f'----Hay {colaN2} aviones esperando a ser trasladados----') # ESTO NO SÉ SI ESTÁ BIEN
        

    if esperando_a_aterrizar > 0: #ahora que se han liberado pistas, atendemos a la cola de aterrizaje
        Y = t_aterrizaje() #enero tiempo que tardará en aterrizar
        LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + Y #registro el tiempo en el que saldrá el avión del nodo 1
        pistasOcupadas += 1
        esperando_a_aterrizar -= 1
        
    elif esperando_a_despegar > 0:
        W = t_salidas() #NO HA ENTRADO? 
        LISTA['tSS'][LISTA['tSS'].index(10E9)] = t + W 
        pistasOcupadas +=1
        esperando_a_despegar -=1 #????
        

finaliza el traslado y espera a poder desembarcar/embarcar

In [ ]:
def rutina_desembarque(t_suc): 
    # avion en aeropuerto: desembarcan pasajeros, embarcan los nuevos y traslado a pistas
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_despegar, puertas_ocupadas
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje, coches_disponibles, nPista
    global esperando_a_aterrizar, colaN3, colaN4
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    t=t_suc #Actualizamos tº de simulación
    
    coches_ocupados -=1 #libero un vehículo
    
    n2 -=1 #Decrementamos el nº de aviones del nodo 2 porque el avión sale del nodo 2.
    n3 +=1 #Incrementamos el nº de aviones en nodo 3
    
    
    #primero un if cola vehiculos? (manejar cola del nodo anterior (N2))
    if coches_ocupados < nCoches and colaN2 > 0: #si tengo aviones pendientes de ser trasladados
        print(f'---Tengo {colaN2} aviones pendientes de traslado')
        Y = t_traslado()
        LISTA['tS2'][LISTA['tS2'].index(10E9)] = t + Y
        print(f'---Programamos el traslado del avión que llegará en el instante {t + Y} al embarque')
        colaN2-=1
        coches_ocupados += 1
        print(f'{colaN2} aviones en espera y {nCoches - coches_ocupados} coches disponibles')
        
    #si hay puertas de embarque disponibles
    if puertas_ocupadas < nPuertas:
        puertas_ocupadas +=1 #tambien lo puedo hacer con num vehiculos
        Z = t_desembarques()
        print(f'----Comienza el cambio de pasajeros y traslado a zona de maniobra que durará {Z}----')
        print(f'----Ahora hay {nPuertas - puertas_ocupadas} puertas disponibles')
        LISTA['tS3'][LISTA['tS3'].index(10E9)] = t + Z
        print(f'---El avion finalizará el embarque en {Z} mins, en el instante {t + Z}')
        
    else: #si no hay puertas disponibles 
        print('----no hay puertas de embarque disponibles---')
        colaN3+=1 #esperando puerta embarque


finaliza el embarque y espera a poder despegar (vuelta al nodo 1)

In [ ]:
def esperando_torre_control(t_suc): 
    # sale del nodo 3 y espera a poder salir del sistema (nodo SS)    
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_despegar, puertas_ocupadas
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje, coches_disponibles, nPista
    global colaN3, nSS, n4, esperando_a_aterrizar, colaN4
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    t=t_suc #Actualizamos tº de simulación
    puertas_ocupadas -=1 #libero un puerta
    
    n3 -=1 #Decrementamos el nº de aviones del nodo 2 porque el avión sale del nodo 2.
    n4 +=1 #Incrementamos el nº de aviones que van a salir del sistema (nodo 4)
    
    # Manejo la cola del nodo anterior
    # si quedan aviones esperando a desembarcar/embarcar
    if puertas_ocupadas < nPuertas and colaN3 > 0:
        print(f'---Tengo {colaN3} aviones pendientes de desembarque')
        Y = t_desembarques()
        LISTA['tS3'][LISTA['tS3'].index(10E9)] = t + Y
        print(f'---Programamos el desembarque/embarque del avión que finalizará en el instante {t + Y}')
        colaN3-=1
        puertas_ocupadas += 1
        print(f'{colaN3} aviones en espera y {nPuertas - puertas_ocupadas} puertas disponibles')
        
    #si hay pistas disponibles y no hay cola de aterrizaje
    if (pistasOcupadas < nPistas) and (esperando_a_aterrizar == 0):
        #generamos tiempo de asignación de pista
        Z = t_salidas()
        print(f'----Comienza la asignación de pista y permiso de despegue que durará {Z}----')
        pistasOcupadas +=1 #ocupo la pista
        print(f'----Ahora hay {nPistas - pistasOcupadas} pistas disponibles')
        LISTA['tSS'][LISTA['tSS'].index(10E9)] = t + Z
        print(f'---El avion finalizará el despegue en {Z} mins, en el instante {t + Z}')
                
    # hay pistas disponibles pero hay cola de aterrizaje    
    elif (pistasOcupadas < nPistas) and (esperando_a_aterrizar > 0):
        #aterrizo el avion que está esperando
        W = t_aterrizaje()
        # Asigna el tiempo de aterrizaje en tS1
        if 10E9 in LISTA['tS1']:
            LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + W
            pistasOcupadas += 1
            print(f'Avion finalizará aterrizaje en el instante {t+W}')
            print(f'Hay {pistasOcupadas} pistas ocupadas y {esperando_a_aterrizar} aviones esperando para aterrizar')
        else:
            print('Error: No se encontró un tiempo libre en tS1 para el aterrizaje.')

        # LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + W
        # pistasOcupadas+=1
        # print(f'Hay {pistasOcupadas} pistas ocupadas y {esperando_a_aterrizar} aviones esperando para aterrizar')
        
    elif (pistasOcupadas == nPistas):    
        print('----no hay pista disponible---')
        esperando_a_despegar+=1 #esperando para el despegue

avión sale del sistema

In [ ]:
def rutina_salida_sistema(t_suc): 
    # sale del nodo 3 y espera a poder salir del sistema (nodo SS)    
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_despegar, puertas_ocupadas
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje, coches_disponibles, nPista
    global colaN3, nSS, n4, esperando_a_aterrizar, colaN4
    
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)

    t=t_suc #Actualizamos tº de simulación
    pistasOcupadas -=1 #avión abandona la pista
    
    # n4 -=1 #Decrementamos el nº de aviones del nodo 2 porque el avión sale del nodo 2.    
    
    # Manejo la cola del nodo anterior
    # si quedan aviones esperando a despegar, y pistas disponibles para ello
    if pistasOcupadas < nPistas and esperando_a_despegar > 0:
        print(f'---Tengo {esperando_a_despegar} aviones pendientes de despegar')
        Y = t_salidas()
        LISTA['tSS'] = t + Y
        print(f'---Programamos la salida del avión que será en el instante {t + Y}')
        esperando_a_despegar-=1
        pistasOcupadas += 1
        print(f'{esperando_a_despegar} aviones en espera y {nPistas - pistasOcupadas} pistas disponibles')
        

In [ ]:

def simul_main():
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_despegar, esperando_a_aterrizar, puertas_ocupadas
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje, nPistas, coches_disponibles
    global esperando_a_aterrizar, colaN3, colaN4
    t = 0                   # Variable reloj. tiempo asíncrono.
    n1 = n2 = n3 = 0        # Variables de estado nº clientes
    NLL1 = NLL2 = NLL3 = 0  # Variables contador nº llegadas
    NS1 = NS2 = NS3 = 0     # Variables contador nº salidas

    nPistas = 3
    coches_disponibles = 20
    
    # Lista de tiempos de eventos
    # LISTA = {}
    # LISTA['tLL1'] = LISTA['tS1'] = LISTA['tS2'] = LISTA['tS3'] = LISTA['tSS'] = -1

    # Arrays for arrival and departure times
    LL1 = []
    LL2 = []
    LL3 = []
    S1 = []
    S2 = []
    S3 = []
    
    LISTA = {
    'tLL1': 10E9,   # Inicializa 'tLL1' como una lista vacía
    'tS1':  [10E9]*nPistas,
    'tS2':  [10E9]*nCoches,
    'tS3':  [10E9]*nPuertas,
    'tSS':  [10E9]*nPistas
}


    n_med_n1 = n_med_n2 = n_med_n3 = 0 #nº medio de pistas ocupadas
    nmed_pistasOcupadas = 0
    coches_ocupados = 0
    puertas_ocupadas = 0

    n_cola_aterrizaje = 0 # Contador de aviones esperando para aterrizar
    n_cola_traslado = 0
    pistasOcupadas = 0
    esperando_a_despegar = 0
    esperando_a_aterrizar = 0

    colaN2 = 0
    colaN3 = 0
    colaN4 = 0
    
    #variables de estado
    todas_las_pistas_ocupadas = True
    puertas_embarque_disponibles = True

    X = t_llegada(0)
    rutina_aterrizaje(X)
    while (t <= T and (LISTA['tLL1'] != 10E9 or LISTA['tS1'] != 10E9 or LISTA['tS2'] != 10E9 or LISTA['tS3'] != 10E9 or LISTA['tSS'] != 10E9)):
        print(LISTA)
        min_key = min(
            LISTA.keys(), 
            key=lambda k: min(LISTA[k]) if isinstance(LISTA[k], list) else LISTA[k]
        )
        if(min_key=='tLL1'): # Si el próximo evento es un aterrizaje (LLN1)
            print('----Ahora toca una llegada----')
            t_suc=LISTA['tLL1']
            LISTA['tLL1']= 10E9
            rutina_aterrizaje(t_suc)
            print(LISTA)
            print()
            
        elif(min_key=='tS1'): # Si el próximo evento es un traslado (SN1)
            print('----Ahora toca un traslado----')
            t_suc=min(LISTA['tS1'])
            LISTA['tS1'][LISTA['tS1'].index(min(LISTA['tS1']))] = 10E9
            rutina_traslado(t_suc)
            print(LISTA)
            print()
            
        elif(min_key=='tS2'): # Si el próximo evento es un deembarque (SN2)
            print('----Ahora toca un cambio de pasajeros----')
            t_suc=min(LISTA['tS2'])
            LISTA['tS2'][LISTA['tS2'].index(min(LISTA['tS2']))] = 10E9
            rutina_desembarque(t_suc)
            print(LISTA)
            print()
            
        elif(min_key=='tS3'): # Si el próximo evento es un deembarque (SN3)
            print('----Ahora toca esperar a torre de control----')
            t_suc=min(LISTA['tS3'])
            LISTA['tS3'][LISTA['tS3'].index(min(LISTA['tS3']))] = 10E9
            rutina_desembarque(t_suc)
            print(LISTA)
            print()
            
        elif(min_key=='tSS'): # Si el próximo evento es un deembarque (SN4)
            print('----Ahora toca esperar a torre de control----')
            t_suc=min(LISTA['tSS'])
            LISTA['tSS'][LISTA['tSS'].index(min(LISTA['tSS']))] = 10E9
            rutina_salida_sistema(t_suc)
            print(LISTA)
            print()
    

In [ ]:
simul_main()

In [ ]:
print(n3)
print(n2)
print(n1)


In [ ]:
print(n_med_n1)
print(n_med_n2)
print(n_med_n3)
print(nmed_pistasOcupadas)